In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import *
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from IPython.display import display
from PIL import Image
import time
from keras.utils import multi_gpu_model
from keras.applications import Xception


t0 = time.time() # initial time

    # dimensions of our images.
img_width, img_height = 150, 150

    # image directories 
train1_data_dir = 'data/train1'
train2_data_dir = 'data/train2'
train3_data_dir = 'data/train3'
val1_data_dir = 'data/val1'
val2_data_dir = 'data/val2'
val3_data_dir = 'data/val3'

    #number of samples
nb_train_samples = 999
nb_validation_samples = 501
epochs = 50
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

    #multi-gpu configuration
model = Xception(weights=None, input_shape=input_shape, classes=2)

model = multi_gpu_model(model, gpus=2)

    # three sub-layers
model1 = Sequential()
model1.add(Conv2D(32, (3, 3), input_shape=input_shape))
model1.add(Activation('relu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))

model2 = Sequential()
model2.add(Conv2D(32, (3, 3), input_shape=input_shape))
model2.add(Activation('relu'))
model2.add(MaxPooling2D(pool_size=(2, 2)))

model3 = Sequential()
model3.add(Conv2D(32, (3, 3), input_shape=input_shape))
model3.add(Activation('relu'))
model3.add(MaxPooling2D(pool_size=(2, 2)))

    # combine the layers 
mergedOut = Add()([model1.output,model2.output,model3.output])

mergedOut = Flatten()(mergedOut)    
mergedOut = Dense(64, activation='relu')(mergedOut)
mergedOut = Dropout(.5)(mergedOut)
mergedOut = Dense(2, activation='sigmoid')(mergedOut)

model = Model([model1.input,model2.input,model3.input], mergedOut)

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


    # this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

    # this is the augmentation configuration we will use for testing:
    # only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

    # function for multiple inputs 
def generate_generator_multiple(generator,dir1, dir2, dir3, batch_size, img_height,img_width):
    genX1 = generator.flow_from_directory(dir1,
                                          target_size = (img_height,img_width),
                                          class_mode = 'categorical',
                                          batch_size = batch_size,
                                          shuffle=False, 
                                          seed=7)
    
    genX2 = generator.flow_from_directory(dir2,
                                          target_size = (img_height,img_width),
                                          class_mode = 'categorical',
                                          batch_size = batch_size,
                                          shuffle=False, 
                                          seed=7)
    
    genX3 = generator.flow_from_directory(dir3,
                                          target_size = (img_height,img_width),
                                          class_mode = 'categorical',
                                          batch_size = batch_size,
                                          shuffle=False, 
                                          seed=7)
    
    while True:
            X1i = genX1.next()
            X2i = genX2.next()
            X3i = genX3.next()
            yield [X1i[0], X2i[0], X3i[0]], X3i[1]  #Yield both images and their mutual label
            
            
train_generator=generate_generator_multiple(generator=train_datagen,
                                           dir1=train1_data_dir,
                                           dir2=train2_data_dir,
                                           dir3=train3_data_dir,
                                           batch_size=batch_size,
                                           img_height=img_height,
                                           img_width=img_height)       


validation_generator=generate_generator_multiple(generator=test_datagen,
                                          dir1=val1_data_dir,
                                          dir2=val2_data_dir,
                                          dir3=val3_data_dir,
                                          batch_size=batch_size,
                                          img_height=img_height,
                                          img_width=img_height) 

    #fit the model to generated (augmented) data 
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

model.save('para_try.h5')

t1 = time.time()

total = t1-t0
    
print(total)


